Для выполнения задания нужно майнить данные. Для этого надо обращаться к автору за данными.

В данных для предикта всего 12 марок авто: *BMW, VOLKSWAGEN, NISSAN, MERCEDES, TOYOTA, AUDI, MITSUBISHI, SKODA, VOLVO, HONDA, INFINITI, LEXUS* по ним и будем парсить данные.

Какие поля нужны: Тип кузова, производитель, цвет, объём двигателя, мощность в ЛС, тип топлива, пробег, кол-во дверей, дата производства, id объявления, тип кпп, регион марки(япония, европа), кол-во владельцев, оригинальный ПТС? тип привода, положение руля, цена

План на парсинг:
*    Создаём массив нужных марок.
*    Обращаемся к странице с объявлениями об авто
*    На страницах с объявлениями в теге `<script id="initial-state" .... type="application/json">` блок `listing: { ... "offers": [{ ` содержит нужную нам информацию. Его нужно распарсить по каждой машине.


In [1]:
from google.colab import files
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import json
import glob

In [2]:
#создаём метод, который объединяет 95% работы в один блок. На вход массив брендов по которым собираем информацию и страница ОТ и ДО
def parse_autoru(brands, pagefrom= 1, pageto= 50):
  #создаём датафрейм с нужными колонками
  parsed = pd.DataFrame(columns=['bodyType','brand','color','engineDisplacement','enginePower','fuelType', 'mileage','numberOfDoors',
                        'productionDate','sell_id','vehicleTransmission','vendor', 'owners','pts', 'gear_type', 'steering_wheel', 'price'])

  #цикл для каждого из переданных брендов
  for brand in brands:
    print('Начали парсить', brand, '...')
    pagestart = pagefrom #странная строчка, нужно чтобы цикл каждый раз начинался со страницы которую передал пользователь

    while pagestart <= pageto: #до 50 страницы
      url = 'https://auto.ru/cars/'+brand+'/all/?page='+str(pagestart) #сформировали url для запроса
      response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}) #кинули запрос на автору
      
      response.encoding = "utf-8"
      page = bs(response.content, 'html.parser') #получили всю страницу
      
      try: #бывают ошибки при получении JSON настранице, так что выполяем в блоке try
        allParamsJson = json.loads(page.find('script', id="initial-state").string) #получаем JSON на странице со всеми параметрами автомобилей
      except: #если не вышло, сохраним фрейм
          print('End. Total parsed:', len(parsed))
          return(parsed) #возвращаем датафрейм
      else: #или начинаем обрабатывать
        for car in allParamsJson["listing"]["data"]["offers"]: #и циклом для каждого авто все параметры в словарь
          new_row = {'bodyType' : car["vehicle_info"]["configuration"].get("human_name","")
                    ,'brand': car["vehicle_info"]["mark_info"]["code"]
                    ,'color': car["color_hex"]
                    ,'engineDisplacement': car["vehicle_info"]["tech_param"]["displacement"]
                    ,'enginePower': car["vehicle_info"]["tech_param"]["power"]
                    ,'fuelType': car["vehicle_info"]["tech_param"]["engine_type"]
                    ,'mileage': car["state"]["mileage"]
                    ,'numberOfDoors': car["vehicle_info"]["configuration"]["doors_count"]
                    ,'productionDate': car["documents"]["year"]
                    ,'sell_id': car['saleId']
                    ,'vehicleTransmission': car["vehicle_info"]["tech_param"]["transmission"]
                    ,'vendor': car['vehicle_info']['vendor']
                    ,'owners': car["documents"].get("owners_number", 0) #в некоторых случаях нет владельцев, новые авто?
                    ,'pts': car["documents"].get("pts_original", False) #и не указано наличие ПТС
                    ,'gear_type': car["vehicle_info"]["tech_param"]["gear_type"]
                    ,'steering_wheel': car["vehicle_info"]["steering_wheel"]
                    ,'price':car["price_info"].get("RUR", 0)} #и нет цены
          #добавляем новую строчку в датафрейм
          parsed = parsed.append(new_row, ignore_index=True)
      pagestart +=1
  print('End. Total parsed:', len(parsed))
  return(parsed) #возвращаем датафрейм

In [3]:
brands = ['BMW', 'VOLKSWAGEN', 'NISSAN', 'MERCEDES', 'TOYOTA', 'AUDI', 'MITSUBISHI', 'SKODA', 'VOLVO', 'HONDA', 'INFINITI', 'LEXUS'] #список брендов
results_df = parse_autoru(brands, 50, 100) #вызываем функцию

results_df.to_csv('parsed.csv', index=False) #сохраняем ДФ

Начали парсить BMW ...
Начали парсить VOLKSWAGEN ...
Начали парсить NISSAN ...
Начали парсить MERCEDES ...
Начали парсить TOYOTA ...
Начали парсить AUDI ...
Начали парсить MITSUBISHI ...
Начали парсить SKODA ...
Начали парсить VOLVO ...
Начали парсить HONDA ...
Начали парсить INFINITI ...
Начали парсить LEXUS ...
End. Total parsed: 20806


Код выше запускаем пачками по 25 страниц, парсим n-раз. Вручную сохранил и перенёс на гугл диск для работы в Colab далее.

Прочитаем файлы, объединим их в один датафрейм и далее будем работать с ним.

In [4]:
#пустой массив
appended_data = []

#циклом читаем всё что попадает под условие parsed*.csv и добавляем в один огромный датафрейм
for path in glob.glob("/content/drive/My Drive/Colab Notebooks/car_price/parsed*.csv"):
  data = pd.read_csv(path, sep=',')
  appended_data.append(data)
    
result = pd.concat(appended_data)
result.head()

,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,numberOfDoors,productionDate,sell_id,vehicleTransmission,vendor,owners,pts,gear_type,steering_wheel,price
0,Седан,BMW,0000CC,2979,231,GASOLINE,225000,4,2003,1106147691-38665f2c,MECHANICAL,EUROPEAN,4,False,REAR_DRIVE,LEFT,700000
1,Внедорожник 5 дв.,BMW,0000CC,1998,192,GASOLINE,95000,5,2019,1106546953-d8199219,AUTOMATIC,EUROPEAN,1,True,ALL_WHEEL_DRIVE,LEFT,3080000
2,Седан,BMW,97948F,1998,184,GASOLINE,5000,4,2021,1106289403-d7bf1d02,AUTOMATIC,EUROPEAN,1,True,ALL_WHEEL_DRIVE,LEFT,5020000
3,Внедорожник 5 дв.,BMW,040001,2993,265,DIESEL,29000,5,2019,1106243521-b287283f,AUTOMATIC,EUROPEAN,2,True,ALL_WHEEL_DRIVE,LEFT,9880000
4,Внедорожник 5 дв.,BMW,040001,2993,340,DIESEL,0,5,2021,1106611443-7880a32b,AUTOMATIC,EUROPEAN,0,True,ALL_WHEEL_DRIVE,LEFT,12740000


In [5]:
print('Всего строк:',len(result)) 
print('Всего уникальных:', len(result.sell_id.unique()))

Всего строк: 473166
Всего уникальных: 86133


In [6]:
#Удалим дубли
result.drop_duplicates(subset=['sell_id'], inplace=True)

#Удалим пустые цены
result.drop(result[result.price < 50].index, inplace=True)

### Теперь приведём данные к единому виду.

Цвета заданы в hex-color надо привести к английским наименованиям

In [7]:
result.color.unique()

array(['0000CC', '97948F', '040001', 'FAFBFB', '200204', '007F00',
       'C49648', '4A2197', '22A0F8', 'EE1D19', 'CACECB', 'FF8649',
       'DEA522', '660099', 'FFD600', 'FFC0CB'], dtype=object)

In [8]:
#цвета в eng
color_dict = {'040001': 'black',
                  'FAFBFB': 'white',
                  '97948F': 'gray' ,
                  '200204': 'brown',
                  '0000CC': 'blue',
                  'CACECB': 'silver',
                  '660099': 'purple', 
                  '007F00': 'green',
                  'FF8649': 'orange',
                  'EE1D19': 'red',
                  '22A0F8': 'light blue',
                  'C49648': 'beige',
                  '4A2197': 'violet',
                  'FFD600': 'yellow',
                  'DEA522': 'gold',
                  'FFC0CB': 'pink' }


result['color'] = result['color'].map(color_dict)

Теперь тип кузова

In [9]:
result.bodyType.unique()

array(['Седан', 'Внедорожник 5 дв.', 'Хэтчбек 5 дв.',
       'Лифтбек Gran Turismo', 'Купе', 'Седан Gran Coupe', 'Седан Long',
       'Универсал 5 дв.', 'Лифтбек Gran Coupe', 'Кабриолет',
       'Хэтчбек 3 дв.', 'Седан 2 дв.', 'Компактвэн Gran Tourer',
       'Родстер', 'Хэтчбек 3 дв. Compact', 'Компактвэн', 'Минивэн Long',
       'Минивэн', 'Пикап Двойная кабина Double Cab', 'Лифтбек', 'Фургон',
       'Фургон Kasten', 'Внедорожник 5 дв. Allspace', 'Минивэн SWB',
       'Пикап Двойная кабина', 'Компактвэн Life Maxi', 'Компактвэн Life',
       'Компактвэн Maxi', 'Минивэн LWB', 'Универсал 5 дв. Alltrack',
       'Фургон Maxi', 'Седан Latio', 'Универсал 5 дв. California',
       'Внедорожник 3 дв.', 'Минивэн L', 'Купе-хардтоп', 'Минивэн XL',
       'Минивэн Extralong', 'Минивэн L3', 'Минивэн L2', 'Минивэн L1',
       'Универсал 5 дв. All-Terrain', 'Внедорожник открытый',
       'Минивэн Extra-long', 'Седан Japan Market',
       'Универсал 5 дв. Fielder', 'Седан-хардтоп',
       'Пикап Дв

Очень много вариантов. Думаю что можно вырезать первое слово из всех наименований, привести в нижний регистр и уже с ним работать.

In [10]:
result['bodyType'] = result['bodyType'].apply(lambda x: x.split(' ')[0].lower())
result.bodyType.unique()

array(['седан', 'внедорожник', 'хэтчбек', 'лифтбек', 'купе', 'универсал',
       'кабриолет', 'компактвэн', 'родстер', 'минивэн', 'пикап', 'фургон',
       'купе-хардтоп', 'седан-хардтоп', 'микровэн', 'лимузин'],
      dtype=object)

In [11]:
pts_dict = { True: 1,
              False: 0}

#маппим и заменяем на eng
result['pts'] = result['pts'].map(pts_dict)

Значительно лучше. Смотрим что вышло.

In [12]:
result.head()

,bodyType,brand,color,engineDisplacement,enginePower,fuelType,mileage,numberOfDoors,productionDate,sell_id,vehicleTransmission,vendor,owners,pts,gear_type,steering_wheel,price
0,седан,BMW,blue,2979,231,GASOLINE,225000,4,2003,1106147691-38665f2c,MECHANICAL,EUROPEAN,4,0,REAR_DRIVE,LEFT,700000
1,внедорожник,BMW,blue,1998,192,GASOLINE,95000,5,2019,1106546953-d8199219,AUTOMATIC,EUROPEAN,1,1,ALL_WHEEL_DRIVE,LEFT,3080000
2,седан,BMW,gray,1998,184,GASOLINE,5000,4,2021,1106289403-d7bf1d02,AUTOMATIC,EUROPEAN,1,1,ALL_WHEEL_DRIVE,LEFT,5020000
3,внедорожник,BMW,black,2993,265,DIESEL,29000,5,2019,1106243521-b287283f,AUTOMATIC,EUROPEAN,2,1,ALL_WHEEL_DRIVE,LEFT,9880000
4,внедорожник,BMW,black,2993,340,DIESEL,0,5,2021,1106611443-7880a32b,AUTOMATIC,EUROPEAN,0,1,ALL_WHEEL_DRIVE,LEFT,12740000


Выглядит как то что нам надо. Сохраним всё в отдельную CSV для дальнейшей работы.

In [13]:
result.to_csv('train.csv', index=False) #сохраняем ДФ